data source: ./simplified_data/data_v2_grouped_symptom.csv



data 處理:

data v2 刪掉所有文字input, deleted 回診,西藥, 刪除非病癥/病, 人手group起相似col



## train 方法:

train with x that only has sympton, model layer: 16-32-32-2, epoch=100, batch_size=32,  learning rate: 0.001, activation=relu, num_med:all, del_med_under_thres=0

## ready to train 但未train



In [1]:

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.utils.class_weight import compute_sample_weight




In [2]:
import numpy as np
import pandas as pd
import random
import json
import statistics
from tabulate import tabulate

# custom import 
from utility_file import my_utilities as myutil
from utility_file import load_data

## Read Data

1. 決定是否要delete 少於某threshold的藥
2. 用load_data裡的`load_data_for_n_med` load data
3. 如只train 頭n 個藥-> set `only_train_1st_n`

In [4]:
del_med_under_thres = 0     # 於所有medical cases中出現次數少於此數的藥->整col 刪除
                            # if set to 250 -> will leave 10 medicine

total_med = 102
only_train_1st_n = None

(X_np, X_val_np, train_y, val_y, 
  num_col_x, num_1_valy, num_0_valy) = load_data.load_data_for_1_med_with_debug(del_med_thres=del_med_under_thres, 
                                                                                n=only_train_1st_n, 
                                                                                file_name='simplified_data/data_v2_grouped_symptom.csv')

assert(isinstance(X_np, np.ndarray))
assert(isinstance(X_val_np, np.ndarray))
assert(isinstance(train_y, pd.DataFrame))
assert(isinstance(val_y, pd.DataFrame))
assert(isinstance(num_col_x, int))


in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (674 rows, 150 cols).
SplitXY:
Shape of X = (673 rows, 46 cols). First 10 data of X:
    乳癌  肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤
1    0   0    0   0   0     0   0    0   0   0
2    0   0    0   0   0     0   0    0   0   0
3    0   0    0   0   0     0   0    0   0   0
4    0   1    0   0   0     0   0    0   0   0
5    0   0    0   0   0     0   0    0   0   0
6    0   0    0   0   0     0   0    0   0   0
7    0   0    0   0   0     0   0    0   0   0
8    0   0    0   0   0     0   0    0   0   0
9    0   0    0   0   0     0   0    0   0   0
10   1   1    0   0   0     0   1    0   0   0
Shape of y = (673 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   0   0   0   0   0   0   0   0   0
2    0   0   0   0   0   0   0   0   0   0
3    1   0   0   0   1   0   0   0   0   0
4    0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   0   0   0   0   0   0
6    

In [ ]:
# data type checking  you can run this if you suspect data type 
# else can skip this cell


# myutil.print_df(val_y, "---- y ----")
# print(val_y)
# checking
# 1. Count occurrence of na, int, float, str in y
na_count = val_y.isna().sum().sum()
str_count = val_y[val_y.map(type) == str].count().sum()
int_count = val_y[val_y.map(type) == int].count().sum()
float_count = val_y[val_y.map(type) == float].count().sum()

print(f"Number of NA values in y: {na_count}")
print(f"Number of str values in y: {str_count}")
print(f"Number of int values in y: {int_count}")
print(f"Number of float values in y: {float_count}")

## compute class weight

In [ ]:

# # class_weights = compute_sample_weight(class_weight='balanced', y=train_y)
# # class_weight_dict = dict(enumerate(class_weights))

# # print(class_weight_dict)

# train_y_np = np.array(train_y)

# num_labels = train_y_np.shape[1]
# class_weight_dic = {}

# for i in range(num_labels):
#     unique_values, counts = np.unique(train_y_np[:, i], return_counts=True)
#     value_frequency_dict = dict(zip(unique_values, counts))
#     total = value_frequency_dict[0] + value_frequency_dict[1]
#     class_weight_dic[i] = {0: (value_frequency_dict[1] / total), 1: 5 * (value_frequency_dict[0] / total)} 
#     # f1: 0.35
#     # class_weight_dic[i] = {0: (total / value_frequency_dict[0]), 1: 5 * (total / value_frequency_dict[1])}

# print(class_weight_dic)
# class_weight_list = np.array(class_weight_dic)

## Build Model

In [ ]:
model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=32, activation='relu'), 
    Dense (units=2, activation='softmax')
])
model.summary()

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


## Train Model

train medicine-by-medicine

In [ ]:

result_df_dict={}   # dict of df
#loss_acc_dict={}    # dict of loss and accuracy of each medicine
accuracy_dict={}    # dict of accuracy of each medicine
raw_prediction_train_dict={}   # dict of raw prediction of train set
raw_prediction_val_dict={}   # dict of raw prediction of val set
histories={}

for i in range(train_y.shape[1]):
#for i in range(2):

    chosen_col = train_y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    print(f"processing the {i+1} of {train_y.shape[1]} medicine: { chosen_col.name}")

    chosen_y_np = chosen_col.values.astype('float64')

    y_val_chosen_col = val_y.iloc[:,i].copy()
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))
    #y_val_chosen_col_np = y_val_chosen_col.values.astype('float64')

    
    # Early stop
    #early_stopping = EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)

    
    # fit model for this medicine
    history =  model.fit(
        x=X_np,
        y=chosen_y_np,
        # class_weight=class_weight_dic[i],
        epochs=100,
        shuffle=True,
        verbose=0,
        #callbacks=[early_stopping]
    )   # batch_size=32 if not specified
    
    # make prediction for this medicine
    predictions_val_set = model.predict(X_val_np)
    if(i==1):
        print("type of predictions val: " ,type(predictions_val_set))
        print("predictions.shape:", predictions_val_set.shape)
        
    raw_prediction_val_dict[chosen_col.name] = predictions_val_set  # save raw result np of val set to dict
    

    predictions_train_set = model.predict(X_np)   # predict against training set for diagonse overfit or underfit
    raw_prediction_train_dict[chosen_col.name] = predictions_train_set  # save raw result np of train set to dict
    histories[chosen_col.name] = history.history    # save history 

print("training done.")


In [ ]:
print( type(history))

In [ ]:
# simple check
# total_zeros = np.sum(predictions_val_set == 0)
# print("total no. of 0 in prediction of val", total_zeros)
# total_ones = np.sum(predictions_val_set == 1)
# print("total no. of 1 in prediction of val", total_ones)
# total_zeros = np.sum(predictions_train_set == 0)
# print("total no. of 0 in prediction of train", total_zeros)
# total_ones = np.sum(predictions_train_set == 1)
# print("total no. of 1 in prediction of train", total_ones)
# print(np.count_nonzero(predictions_val_set))
# print(np.count_nonzero(predictions_train_set))


## Handle result

In [ ]:
# cal TP, FP, TN, FN for train set
total_tp_train=0
total_fp_train =0
total_tn_train =0
total_fn_train =0

predicted_value_index = None
ground_truth_index = None

for key, arr in raw_prediction_train_dict.items():
    df_tmp = pd.DataFrame(arr, columns=["predicted as 0", "predicted as 1"])
    df_tmp["predicted value"] = np.where(df_tmp["predicted as 0"] > df_tmp["predicted as 1"], 0, 1)
    col_num = train_y.columns.get_loc(key)
    df_tmp["ground truth"] = train_y.iloc[:,col_num].copy().values
    if predicted_value_index is None:
        predicted_value_index = df_tmp.columns.get_loc('predicted value')
    if ground_truth_index is None:
        ground_truth_index = df_tmp.columns.get_loc('ground truth')

    total_tp_train += ((df_tmp['ground truth'] == 1) & (df_tmp['predicted value'] == 1)).sum()
    total_fp_train += ((df_tmp['ground truth'] == 0) & (df_tmp['predicted value'] == 1)).sum()
    total_fn_train += ((df_tmp['ground truth'] == 1) & (df_tmp['predicted value'] == 0)).sum()
    total_tn_train += ((df_tmp['ground truth'] == 0) & (df_tmp['predicted value'] == 0)).sum()


overall_f1_train = 2 * total_tp_train / (2 * total_tp_train + total_fp_train + total_fn_train) if (2 * total_tp_train + total_fp_train + total_fn_train) != 0 else 0

train_set_acc = {}
train_set_acc["TP"]=total_tp_train
train_set_acc["FP"]=total_fp_train
train_set_acc["FN"]=total_fn_train
train_set_acc["TN"]=total_tn_train
# no. of medical case in train set = # row in train_x * # col in train_y or val_y
med_case_train = (X_np.shape[0] *  val_y.shape[1])
train_set_acc["TP_percentage"] = total_tp_train / med_case_train
train_set_acc["FP_percentage"] = total_fp_train / med_case_train
train_set_acc["FN_percentage"] = total_fn_train / med_case_train
train_set_acc["TN_percentage"] = total_tn_train / med_case_train
train_set_acc["overall_f1"] = overall_f1_train


In [ ]:

train_set_acc["precision"] = total_tp_train / (total_tp_train + total_fp_train) if (total_tp_train + total_fp_train) != 0 else 0
train_set_acc["recall"] = total_tp_train / (total_tp_train + total_fn_train) if (total_tp_train + total_fn_train) != 0 else 0

# precision = 判斷為true之中有多少是對的 = TP / (TP + FP) 
# recall  =  實際為true之中有多少被找到  = TP / (TP + FN) 

try:
    assert((total_tp_train + total_fp_train + total_fn_train + total_tn_train) == med_case_train)
except:
    print("assertion error for calculation check of train set")
    print(f"tp+fp+tn+fn={total_tp_train + total_fp_train + total_fn_train + total_tn_train} , total med case={med_case_train}")


In [ ]:


# put prediction into df for val set

for key, arr in raw_prediction_val_dict.items():
    df_predictions = pd.DataFrame(arr, columns=["predicted as 0", "predicted as 1"])
    df_predictions["predicted value"] = np.where(df_predictions["predicted as 0"] > df_predictions["predicted as 1"], 0, 1)
    col_num = val_y.columns.get_loc(key)
    df_predictions["ground truth"] = val_y.iloc[:,col_num].copy().values
    df_predictions["is_correct"] = df_predictions["predicted value"] == df_predictions["ground truth"]
    accuracy = df_predictions["is_correct"].mean()
    accuracy_dict[key] = accuracy  
    result_df_dict[key] = df_predictions
    result_df_dict[key] = df_predictions
    print(tabulate(df_predictions, headers='keys',tablefmt='psql'))

In [ ]:

predicted_value_index = None
ground_truth_index = None

acc_each_med={}
total_tp=0
total_fp=0
total_fn=0
total_tn=0

# df.columns.tolist()= pick list of column names

for i in train_y.columns.tolist(): # build empty dict for holding TP, FP, FN, TN, accuracy, f1_score of each medicine
    acc_each_med[i]={"TP": None, "FP": None, "FN": None, "TN": None, "accuracy": None, "f1_score": None}

# calculate f1 score of for each medicine

for key, df in result_df_dict.items():
    # create 'TP/FP/TN/TN/FN' column

    if predicted_value_index is None:
        predicted_value_index = df.columns.get_loc('predicted value')
    if ground_truth_index is None:
        ground_truth_index = df.columns.get_loc('ground truth')


    df['TP/FP/TN/TN/FN'] = df.apply(lambda row: 'TP' if ( row.iloc[predicted_value_index] and row.iloc[ground_truth_index] ) else 
                                 ('FP' if (row.iloc[predicted_value_index]  and (not row.iloc[ground_truth_index] )) else 
                                  ('FN' if (( not row.iloc[predicted_value_index] ) and row.iloc[ground_truth_index] ) else 
                                   ( 'TN' if ((not row.iloc[predicted_value_index]) and (not row.iloc[ground_truth_index]) ) else 'UN'))), axis=1)

    # count occurrences of 'TP', 'FP', 'FN', and 'TN' of *this medicine*
    counts = df['TP/FP/TN/TN/FN'].value_counts()
    # get number of 'TP', 'FP', 'FN', and 'TN'
    num_tp = counts.get('TP', 0)
    num_fp = counts.get('FP', 0)
    num_fn = counts.get('FN', 0)
    num_tn = counts.get('TN', 0)

    f1_score = 2 * num_tp / (2 * num_tp + num_fp + num_fn) if (2 * num_tp + num_fp + num_fn) > 0 else 0
    acc_each_med[key]["TP"] = num_tp
    acc_each_med[key]["FP"] = num_fp
    acc_each_med[key]["FN"] = num_fn
    acc_each_med[key]["TN"] = num_tn
    acc_each_med[key]["accuracy"] = accuracy_dict[key].item()
    acc_each_med[key]["f1_score"] = f1_score

    total_tp += num_tp
    total_fp += num_fp
    total_fn += num_fn
    total_tn += num_tn

    ### debug messages
    print("in for loop of result_df_dict")
    print(f"processing : {key} ....")
    print(f"TP of {key}: {num_tp}", sep="\t")
    print(f"FP of {key}: {num_fp}", sep="\t")
    print(f"FN of {key}: {num_fn}", sep="\t")
    print(f"TN of {key}: {num_tn}", sep="\t")
    print(f"f1_score of {key}: {f1_score}")
    print(f"total tp now is: {total_tp}"  , sep="\t")
    print(f"total fp now is: {total_fp}"  , sep="\t")
    print(f"total fn now is: {total_fn}"  , sep="\t")
    print(f"total tn now is: {total_tn}"  , sep="\t")

overall_f1 = 2 * total_tp / (2 * total_tp + total_fp + total_fn) if (2 * total_tp + total_fp + total_fn) > 0 else 0
total_med_case =  len(val_y)* val_y.shape[1]
print(" \n*****   end of for loop   *****\n")
print("now doing calculation checking...")
mean_accuracy = statistics.mean(accuracy_dict.values())
acc_each_med["overall"]={}
acc_each_med["overall"]["TP"] = total_tp
acc_each_med["overall"]["FP"] = total_fp
acc_each_med["overall"]["FN"] = total_fn
acc_each_med["overall"]["TN"] = total_tn
acc_each_med["overall"]["TP_percentage"] ={total_tp/total_med_case}
acc_each_med["overall"]["FP_percentage"] ={total_fp/total_med_case}
acc_each_med["overall"]["FN_percentage"] ={total_fn/total_med_case}
acc_each_med["overall"]["TN_percentage"] ={total_tn/total_med_case}
acc_each_med["overall"]["f1_score"]=overall_f1
acc_each_med["overall"]["mean_accuracy"]=mean_accuracy
acc_each_med["overall"]["precision"] = total_tp / (total_tp + total_fp) if (total_tp + total_fp) != 0 else 0
acc_each_med["overall"]["recall"] = total_tp / (total_tp + total_fn) if (total_tp + total_fn) != 0 else 0


try:
    assert((total_tp + total_fp + total_fn + total_tn) == total_med_case)
    assert((total_tp + total_fn)== num_1_valy)  # TP+FN=all 1 in val_y
    assert((total_fp + total_tn)== num_0_valy)  # TN+FP=all 0 in val_y
    
except:
    print("wrong calculation!")
    print(f"otal_tp + total_fp + total_fn + total_tn= {total_tp + total_fp + total_fn + total_tn}", sep="\t")
    print(f"total_med_case= {total_med_case}")
    print(f"total_tp + total_fn= {total_tp + total_fn}", sep="\t")
    print(f"num_1_valy= {num_1_valy}")
    print(f"total_fp + total_tn= {total_fp + total_tn}", sep="\t")
    print(f"num_0_valy= {num_0_valy}")
else:
    print("checking passed")


In [ ]:


print(f"total tp: {total_tp}, \t percentage = {total_tp/total_med_case}" )
print(f"total fp: {total_fp}, \t percentage = {total_fp/total_med_case}" )
print(f"total fn: {total_fn}, \t percentage = {total_fn/total_med_case}" )
print(f"total tn: {total_tn}, \t percentage = {total_tn/total_med_case}" )
print(f"overall f1 score: {overall_f1}")
print("mean accuracy of all medicine: ", mean_accuracy)
print("precision: ",acc_each_med["overall"]["precision"])
print("recall: ", acc_each_med["overall"]["recall"])
print(acc_each_med)         # can't use json.dumps as there are np.int64
# precision = 判斷為true之中有多少是對的 = TP / (TP + FP) 
# recall  =  實際為true之中有多少被找到  = TP / (TP + FN)
print("-----------------------\n-----  training set accuracy  -----")
print(train_set_acc)


In [ ]:

file_path_suffix = "sympton_only_each_med_csv"   # type the dir for you to remember where u save the result
for key, df in result_df_dict.items():
    print(f"DataFrame for {key}:")
    #myutil.print_df(df)
    myutil.df_to_csv(df, save_path=("./result/"+file_path_suffix), file_prefix=key)
   



In [ ]:
# make df for all f1_score of each medicine
all_f1_df = pd.DataFrame([(key, val['f1_score'], (val['TP']+val['FN'])) for key, val in acc_each_med.items()], columns=['medicine', 'f1_score', 'TP+FN'])
myutil.df_to_csv(all_f1_df, save_path=("./result/"+file_path_suffix), file_prefix='all_f1_score')

In [ ]:
# save f1_score and TP/FP/TN/TN/FN

spec_str = "train with x that only has sympton, model layer: 16-32-32-2, epoch=100, batch_size=32,  learning rate: 0.001, activation=relu, num_med:all, del_med_under_thres=0"
# need to type this spec str each time to record the result

file_path="./result/symptom_only/"
myutil.dict_to_txt(acc_each_med, save_path=file_path, 
                   file_prefix="accuracy_each_med",
                   textbox=spec_str)


myutil.dict_to_txt(train_set_acc, save_path=file_path, 
                   file_prefix="accuracy_train_set",
                   textbox="train set"+spec_str)
